# Noisy Recovery 
Adapting code from https://github.com/MLI-lab/noisy_dna_data_storage/blob/master/LSH_clustering.ipynb

3 Step Process

1. Clustering
2. Multiple Sequence Alignment
3. Consensus Decoding

In [1]:
%load_ext autoreload
%autoreload 2

## Reading simulated data from PBSim

In [5]:

import os
from utils import get_reference_from_file, read_strands_from_file_alt

filepath = r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\PBSIM-PacBio-Simulator"


sequenced_strands = {}
collected_files = set()

for file in os.listdir(filepath)[:30]:
    if file[:2] == "sd":
        
        filename = file[:7]

        if filename in collected_files:
            continue

        collected_files.add(filename)

        ref_file = os.path.join(filepath, filename + ".ref")
        fastq_file = os.path.join(filepath, filename + ".fastq")

        ref = get_reference_from_file(ref_file)
        strands = read_strands_from_file_alt(fastq_file)
        sequenced_strands[ref] = strands

print(f"Collected {len(sequenced_strands.keys())} files")
            

Collected 1 files


In [6]:
from creating_synthesized_strands_from_strands import get_original_strands
ids, coupling_rates, capping_flags, original_strands = get_original_strands()

## Clustering

Picking a strand to simulate

In [7]:
strand_id = ids[0]
original_strand = original_strands[strand_id]
seq_strands = sequenced_strands[strand_id]

In [8]:
trimmed_seqs = [seq for seq in seq_strands if len(seq) >= 55 and len(seq)<=200]

Copied code (to be refactored)

In [9]:

import time
import numpy as np
import itertools

#===== assign numbers to shingles of each sequence=====#
def kmerDNA(seq,k=3):
    kmer = []
    for ell in range(len(seq)-k+1):
        nstr = seq[ell:ell+k]
        index = 0
        for j,c in enumerate(nstr):
            if c == 'A':
                i = 0
            elif c == 'C':
                i = 1
            elif c == 'G':
                i = 2
            elif c == 'T':
                i = 3
            else:
                index = -1
                break
            index += i*(4**j)
        kmer += [index]
    return kmer

#=====min-hash object=====#
class minhashsig():
    # min-hash of k-mers
    def __init__(self,m,k):
        # m is the number of signatures
        self.tables = [np.random.permutation(4**k) for i in range(m)]
        self.k = k
    def generate_signature(self,seq):
        kmer = kmerDNA(seq,self.k)
        sig = [ min([table[i] for i in kmer]) for table in self.tables]
        return sig
    
#=====pair detection=====#
def extract_similar_pairs(sigs,m,k_lsh,ell_lsh,maxsig):
    # sigs: minhash signatures
    # ell_lsh: number of LSH signatures
    # k_lsh: number of MH signatures to be concatenated
    # we use generatrs to yield a number of pairs at a time for the sake of memory efficiency
    
    pairs = set([])
    
    # generate ell_lsh random indices
    for ell in range(ell_lsh):
        pair_count = 0
        s = time.time()
        lshinds = np.random.permutation(m)[:k_lsh]
        # generate LSh signatures
        lshsigs = []
        for sig in sigs:
            lshsig = 0
            for i,lshind in enumerate(lshinds):
                lshsig += sig[lshind]*(maxsig**i)
            lshsigs += [lshsig]
        d = {}
        for ind,sig in enumerate(lshsigs):
            if sig in d:
                d[sig] += [ind]
            else:
                d[sig] = [ind]
        for candidates in d.values():
            cent = set([])
            if len(candidates) > 1:
                for pair in itertools.combinations(candidates,2):
                    cent.add(pair[0])
                    if len(cent)==1:
                        pairs.add(pair)
                    else:
                        break
                        
        yield pairs,ell
        pair_count += len(pairs)
        pairs = set([])

#=====form clusters based on pairs=====#
def center_cluster(pairs):
    clusters = {}
    hold = 0
    t_counter = 0
    ell_copy = 0
    pairsize = 0
    while not hold:
        
        try:
            out = next(pairs)
            pairs_sort = list(out[0])
            ell = out[1]
            pairsize += len(pairs_sort)
            pairs_sort.sort()
            s = time.time()
            for (u,v) in pairs_sort:
                if u in clusters:
                    clusters[u] += [v]
        
                if v in clusters:
                    clusters[v] += [u]
        
                if v not in clusters and u not in clusters:
                    clusters[u] = [v]
    
        except StopIteration:
            hold = 1
            print("clustering completed","---",pairsize,"pairs clustered")
        if ell==ell_copy:
            t_counter += time.time()-s
        else:
            print("Clustering time for LSH",ell_copy,":",t_counter,'\n')
            t_counter = time.time()-s
            ell_copy = ell
 
    return clusters

#=====LSH clustering (main function)=====#
def lsh_cluster(seqs,m,k,k_lsh=2,ell_lsh=4):
    # This is the main function
    maxsig = 4**k
    minhash = minhashsig(m,k)
    sigs = [minhash.generate_signature(seq[:40]) for seq in seqs]
    pairs = extract_similar_pairs(sigs,m,k_lsh,ell_lsh,maxsig)
    clusters = center_cluster(pairs)
    return clusters




def filter_nonunique(seqs):
    # filter out sequences that appear many times
    d = {}
    ctr = 0
    for i,seq in enumerate(seqs):
        if seq in d:
            d[seq] += [i]
        else:
            d[seq] = [i]
    import operator
    sorted_d = sorted(d.items(), key=operator.itemgetter(1))
    sorted_d.reverse()
    return d,ctr

TRIVIAL = True # when real synthesized data is selected, set this flag to "True" (this dramatically reduces the runtime)


if TRIVIAL:
    start = time.time()
    nbeg = 14
    d,ctr = filter_nonunique([seq[:nbeg] for seq in trimmed_seqs])
    clusters = [d[a] for a in d if len(d[a]) > 3]
    end = time.time()
    print("Runtime:",round(end-start,1),"s")
    print(len(clusters),"number of clusters created.")
    fclusts = clusters.copy()
else:
    # set up the parameters and call the lsh_cluster function
    k_lsh = 4
    sim = 0.5
    ell_lsh = int(1/(sim**k_lsh))
    m,k=50,5
    start = time.time()
    clusters = lsh_cluster(trimmed_seqs,m,k,k_lsh,ell_lsh)
    end = time.time()

    print("Runtime:",round(end-start,1),"s")
    print(len(clusters),"number of clusters created")

Runtime: 0.0 s
16 number of clusters created.


Will need to filter if I use LSH, but moving forward for now

### Filtering Clusters

In [91]:

from skbio.alignment import local_pairwise_align_ssw
from skbio import DNA

# adding the center of each cluter to the cluster (also removing duplicates from each cluster)
clusts = [ [c] + list(set(clusters[c])) for c in clusters if len(clusters[c]) > 3 ]

#=====max matching=====# 
def max_match(seq1,seq2):
    # This function checks whether seq1 and seq2 are similar or not
    # Checking all pairs within a cluster dramatically increases the time complexity, 
    # so by default, in the next cell, we call this function to only check the pairs
    # that one of their members is the cluster center
    
    alignment,score,start_end_positions \
        = local_pairwise_align_ssw(DNA(seq1) , DNA(seq2) , match_score=2,mismatch_score=-3)
    a = str(alignment[0])
    b = str(alignment[1])
    ctr = 0
    for i,j in zip(a,b):
        if i==j:
            ctr += 1
    return ctr

th = 35 # filtering threshold

k = len(clusts)
s = time.time()
fclusts = []
for i,c in enumerate(clusts):
    cent = reads[c[0]]
    cc = [c[0]]
    for e in c[1:]:
        score = max_match(cent,reads[e])
        if score >= th:
            cc += [e]
    fclusts += [cc]
    if i%1000 == 0:
        print("%",round(i*100/len(clusts),2),"of the clusters are filtered.")
print("filtering time for",k,"clusters:",round(time.time()-s,2),"s")

ModuleNotFoundError: No module named 'skbio'

## Aligning the Clusters

In [12]:

from Bio import AlignIO
import subprocess

def multiple_alignment_muscle(cluster, out=False):
    
    # write cluster to file
    file = open("clm.fasta","w") 
    
    for i,c in enumerate(cluster):
        file.write(">S%d\n" % i)
        file.write(c)
        file.write("\n")
    file.close()

    muscle_exe = r"muscle-windows-v5.2.exe" # assuming you've already put this in the main directory
    output_alignment = "clmout.fasta"


    #!.\muscle-windows-v5.2.exe -align clm.fasta -output clmout.fasta
    # Running Muscle through command line
    subprocess.run(
        args=[
            ".\muscle-windows-v5.2.exe", "-align", "clm.fasta", "-output", "clmout.fasta"
        ],
        check=True
    )

    msa = AlignIO.read(output_alignment, "fasta")
    if out:
        print(msa)
    alignedcluster = []
    for i in msa:
        alignedcluster += [i.seq]
    return alignedcluster

In [13]:

from random import shuffle

fresults = []

def align_clusters(clusters,masize = 15):
    ### align clusters, generate candidates
    for i, clusterinds in enumerate(clusters):
        cluster = [trimmed_seqs[i] for i in clusterinds]
        if len(cluster) < 3:
            continue
        if len(cluster) > masize:
            for j in range(5):
                shuffle(cluster)
                ma = multiple_alignment_muscle(cluster[:masize])
                fresults.append(ma)
        else:
            ma = multiple_alignment_muscle(cluster[:masize])
            fresults.append(ma)
            
        if i % 1000 == 0:
            print("%",round(i*100/len(clusters),2),"of the clusters are aligned.")

In [14]:
s = time.time()
align_clusters(fclusts,15)
print("Alignment Runtime:",round(time.time()-s,2),"s")

% 0.0 of the clusters are aligned.
Alignment Runtime: 3.58 s


In [15]:
fresults

[[Seq('CGCAATGGGCATCAA-CTACCGGCCTCCCCGGGTTCCCGACCGCCATCACATTC...TAT'),
  Seq('CGCAATGGGCATCACACTTACG--GCCCCC-AGGTCCCGACCGGCTCCACTTGC...CAG'),
  Seq('CGCAATGGGCATCAA-CTAC-G--CCTCCCGGGTCCCTGACCCGGCTCACATAT...AAA'),
  Seq('CGCAATGGGCATCAA-CCGACG--CTCCCC-GGGTCCCGACGGAG-TCACATT-...CCA')],
 [Seq('CCTATTTTGTTACAATGT---TTCTGGCCCGAAGGAAGGAATACTTTAA-ACTT...AGT'),
  Seq('CCTATTTTGTTACACACCGTTTTCTG--CCGAGGAAGCGAATACTTGAA--CT-...GGA'),
  Seq('CCTATTTTGTTACAACCGT--TTCTG--CCGAGACGGCGAGACTTTGAAAACT-...GTA'),
  Seq('CCTATTTTGTTACAAACCG--TTCTG--CCGAGTAAGCGAATACTTTGATACT-...GGC'),
  Seq('CCTATTTTGTTACAACCT---TTCTG--CCGATGAGCGAATTCCATGAA-ACTT...GGA'),
  Seq('CCTATTTTGTTACAACCTATTTTCTG--CCGAGGAAGCGTATACTTGAA-ACT-...AGG')],
 [Seq('CGCACATGGGCATCAC---T--ACGGCC-TC-----C-GGG-GTCC-CGA-AC-...TCC'),
  Seq('CGCACATGGGCATCAA--CC--ACGGCT-CC-------CGG-GATCCTGA-C--...GGC'),
  Seq('CGCACATGGGCATCAA--CT--AACGCC-CC-------CGG-TACC-GAA-CCC...CCG'),
  Seq('CGCACATGGGCATCAA--GAT-ACGGAC-CCCTTCAC-CGGGGATC-CGA-T--...GCA'),
  Se

## Majority Merging

In [23]:

import operator

# This function returns the fraction of origignal squences recovered given a number of candidates
def fraction_recovered(candidates,orig_seqs):
    d = {}
    for seq in orig_seqs:
        d[seq] = 0
    for cand in candidates:
        if cand in d:
            d[cand] += 1
    av = sum([ d[seq]>0 for seq in d]) / len(d)
    print("Fraction of recovered sequences: ", av )
    if av>0:
        print("Fraction of recovered sequences: ", sum([ d[seq] for seq in d]) / len(d) / av )

def majority_merge(reads,weight = 0.4):
    # assume reads have the same length
    res = ""
    for i in range(len(reads[0])):
        counts = {'A':0,'C':0,'G':0,'T':0,'-':0,'N':0}
        for j in range(len(reads)):
            counts[reads[j][i]] +=1
        counts['-'] *= weight
        mv = max(counts.items(), key=operator.itemgetter(1))[0]
        if mv != '-':
            res += mv
    return res

candidates = []
for ma in fresults:
    candidates.append(majority_merge(ma,weight=0.5))

In [24]:
candidates

['CGCAATGGGCATCAACTAACGCCCCCCGGGGTCCCGACCGGCATCACATTCCCTTCGTCTGCGGTATTAAACTCTCAACTGTATTCCAGAAAACCAACCCTCCAAACCGCGACCGTTCCGACGGAAGTTCGTATTGGCAGCAGCTTTTACGGTTCACTACTAAGTCCAAGTATTGCTCCTGCCCGCAA',
 'CCTATTTTGTTACAACCTGTTTTCTGCCGAGGAAGCGAATACTTGAAACTTTAGTAGAACCCTAAGCTGTGCCAATACGAAACTTCCGTCCGAACCGGCTGCGGGTTGGAGAGGAGTGCTATCTGAATATACCAGTGGAGAGGTTAAATAACACGAGAGAAAGCAATGTGAGGGCCGGTGGGAGACCGGGGGGA',
 'CGCACATGGGCATCAACTACGGCCTCCCGGGTCCCCGACCCGGCTCACATTGCTTCGTCGTGTTATTTTAAACTCTCACTGTTATTTCAGATAAGCACTACCTCCAACCCGCGACCGGTTCCGACGGAAGTTCGTATTGGCAACAGCTTACGGTTTCTACTAAAGTTCAAGTATTCGCTTCCCCTCCG',
 'CGCACATGGGCATCAACTACGGCCCTCCCGGGGTCCCGACCGGCTCACATTGCTTCGTCGTGTTATTTAAACTCTCACTGTATTTCCAGATAGCACACTACCTCCAACCCGCCGACCGTTCCGACGGAAGTTCGTATTGGCACCAGCTTACGGTTTCTACTAAGTTTTCAAGTATTCGCTTCCCCGCA',
 'CGCACATGGGCATCAACTACGGCCCTCCCGGGTTCCGACCCGGCTCACATTGGCTTCGTCGTGTTATTTAAACTCTCACTGTATTTCAGGAAAGCACTACCTCCAACCGGCGACCGTTCCGACGGAAGTTCGTATTGGCAACAGCTTACGGTTCTTACTAAGGTTTCAAGTATTCGCTTCCCTGGCA',
 'CGCACATGGGCATCAACTACGGCCCCCC

In [ ]:
from aligned_clustering import conduct_align_clustering



for i in range(100):
    result_dict = conduct_align_clustering(
        original_strand=original_strand,
        trimmed_seqs=trimmed_seqs,
        display=True
    )

Starting clustering
Runtime: 0.0 s
16 number of clusters created.
% 0.0 of the clusters are aligned.
Evaluating recovery percentage
Best recovery percentage in candidates = 0.44919786096256686
Starting clustering
Runtime: 0.0 s
16 number of clusters created.
% 0.0 of the clusters are aligned.
Evaluating recovery percentage
Best recovery percentage in candidates = 0.36649214659685864
Starting clustering
Runtime: 0.0 s
16 number of clusters created.
% 0.0 of the clusters are aligned.
Evaluating recovery percentage
Best recovery percentage in candidates = 0.37566137566137564
Starting clustering
Runtime: 0.0 s
16 number of clusters created.
% 0.0 of the clusters are aligned.
Evaluating recovery percentage
Best recovery percentage in candidates = 0.35789473684210527
Starting clustering
Runtime: 0.0 s
16 number of clusters created.
% 0.0 of the clusters are aligned.
Evaluating recovery percentage
Best recovery percentage in candidates = 0.4270833333333333
Starting clustering
Runtime: 0.0 s
1

In [21]:
from utils import get_recovery_percentage

fresults[0][0]

Seq('CGCAATGGGCATCAA-CTACCGGCCTCCCCGGGTTCCCGACCGCCATCACATTC...TAT')

In [22]:
fresults

[[Seq('CGCAATGGGCATCAA-CTACCGGCCTCCCCGGGTTCCCGACCGCCATCACATTC...TAT'),
  Seq('CGCAATGGGCATCACACTTACG--GCCCCC-AGGTCCCGACCGGCTCCACTTGC...CAG'),
  Seq('CGCAATGGGCATCAA-CTAC-G--CCTCCCGGGTCCCTGACCCGGCTCACATAT...AAA'),
  Seq('CGCAATGGGCATCAA-CCGACG--CTCCCC-GGGTCCCGACGGAG-TCACATT-...CCA')],
 [Seq('CCTATTTTGTTACAATGT---TTCTGGCCCGAAGGAAGGAATACTTTAA-ACTT...AGT'),
  Seq('CCTATTTTGTTACACACCGTTTTCTG--CCGAGGAAGCGAATACTTGAA--CT-...GGA'),
  Seq('CCTATTTTGTTACAACCGT--TTCTG--CCGAGACGGCGAGACTTTGAAAACT-...GTA'),
  Seq('CCTATTTTGTTACAAACCG--TTCTG--CCGAGTAAGCGAATACTTTGATACT-...GGC'),
  Seq('CCTATTTTGTTACAACCT---TTCTG--CCGATGAGCGAATTCCATGAA-ACTT...GGA'),
  Seq('CCTATTTTGTTACAACCTATTTTCTG--CCGAGGAAGCGTATACTTGAA-ACT-...AGG')],
 [Seq('CGCACATGGGCATCAC---T--ACGGCC-TC-----C-GGG-GTCC-CGA-AC-...TCC'),
  Seq('CGCACATGGGCATCAA--CC--ACGGCT-CC-------CGG-GATCCTGA-C--...GGC'),
  Seq('CGCACATGGGCATCAA--CT--AACGCC-CC-------CGG-TACC-GAA-CCC...CCG'),
  Seq('CGCACATGGGCATCAA--GAT-ACGGAC-CCCTTCAC-CGGGGATC-CGA-T--...GCA'),
  Se

In [ ]:
get_recovery_percentage()